## Proyecto Final MCPP

#### Tipos de grafos

##### Grafos dirigidos y no dirigidos

La gran diferencia entre estos dos tipos de grafos es que los dirigidos especifican de que forma debo recorrer un grafo, es decir, existe un nodo inicial y uno final; mientras que en un grafo no dirigido no existe un orden específico en recorrer el grafo. (Ver presentación para mayor claridad).

In [25]:
### Librerías que se requieren para el proyecto
import itertools as it
import collections as co
import pickle
import numpy as np

La idea es crear dos clases, la primera yo defino un grafo no dirigido de las distintas estaciones del transmilenio, a su vez en esta clase podre crear los diferentes caminos que puedo tener para llegar de una estación a otra.

Para ello requiero definir diferentes conceptos:
#### Vecinos de un nodo
Son todos los nodos que estan conectados con un nodo específico.
#### Vecindades
Es un diccionario que consta de todos los nodos con sus respectivos vecinos.

##### Específicaciones dentro de la clase: estaciones_transmilenio_no_dirigido

- Los lados o conexiones estan dados por una lista: ["a","b"] en este caso los elementos son nodos, como aquí no importa el orden ["a","b"] es igual a ["b","a"].
- Los nodos los definiré con numeros. Especificaré mas adelante la relacion entre numeros y estaciones

In [12]:
### La clase no dirigido, me permite crear un grafo no dirigdo
class estaciones_transmilenio_no_dirigido:
    '''
    Creo un constructor: Para realizar el grafo este consta de las estaciones de transmilenio que 
    representan los nodos del grafo y los lados que son las conexiones que hay entre las diferentes 
    estaciones
    ''' 
    def __init__(self, nodos, lados):
        self.estaciones=nodos
        self.conexiones=lados
        # Agrego un nuevo atributo que será un diccionario de las vencidades 
        self.vecindades={}
    # Funcion que aplicamos en anteriores proyectos para crear vecindades
    def crearvecindades(self):
        if self.vecindades=={}: 
            # Recorro todas las conexiones posibles en las diferentes estaciones de transmilenio
            for j in self.conexiones:
                # Pregunto si el nodo en la posición 0 esta ya creado en las vecindades
                if j[0] in self.vecindades:
                    a=self.vecindades[j[0]]+(j[1],)
                    del self.vecindades[j[0]]
                    self.vecindades[j[0]]=a
                else:
                    self.vecindades[j[0]]=(j[1],)
                if j[1] in self.vecindades:
                    a=self.vecindades[j[1]]+(j[0],)
                    del self.vecindades[j[1]]
                    self.vecindades[j[1]]=a
                else:
                    self.vecindades[j[1]]=(j[0],)
        # Todos los posibles caminos de un nodo a otro
    def crear_caminos(self,inicial, final):
        # Se requiere de la libreria Collections
        cola=co.deque()
        cola.append((inicial,[inicial]))
        while cola:
            (v,camino)=cola.pop()
            for vertice in set(self.vecindades[v])-set(camino):
                if vertice==final:
                    yield camino+[vertice]
                else:
                    cola.append((vertice,camino+[vertice]))

Aca muestro el funcionamiento de la clase: Creo un grafo con 5 nodos y a su vez la union que hay entre los diferentes nodos.

In [3]:
a=estaciones_transmilenio_no_dirigido([1,2,3,4,5],[(1,2),(3,4),(4,5),(3,1)])
a.crearvecindades()
print(a.vecindades)


{2: (1,), 4: (3, 5), 5: (4,), 3: (4, 1), 1: (2, 3)}


Muestro el diccionario de como se generan las vecindades

Ahora para generar un grafo con mas características realizamos un grafo donde los lados tienen dos importantes aspectos:

- Capacidad: Para explicar este concepto daré un ejemplo: Imaginemos una tubería, esta tiene un máximo de capacidad para que fluya el agua. Para el caso de transmilenio, para que la gente se transporte de una estación a otra el transmilenio cuenta con una capacidad máxima de personas en el trasnporte
- Flujo: Es la cantidad de personas que se encuentran actualmente dentro el transporte

Un apunte redundante es que es necesario tener en cuenta que el flujo no puede ser nunca mayor a la capacidad.

En los lados de un grafo se identifica a un nodo como cabeza o como cola, depende de la dirección del lado

### Algoritmo Ford-Fulkerson

Este algoritmo me permite encontrar la forma óptima de que el flujo corra de manera máxima a través de la red. Para empezar hay que tener en cuenta diferentes aspectos para poder aplicar el algoritmo:

- Fuente: Nodo donde inicia el flujo a correr.
- Desague: Nodo final, donde termina el flujo.

Para el caso del transmilenio, una cierta cantidad de personas parten de una estación y buscan llegar a otra. Entonces la intención de este proyecto es: Primero, generar una simulación aleatoria del funcionamiento del algoritmo; Segundo, poder utilizar la encuesta de movilidad para poder ver como se mueve la gente en la ciudad, es decir, de donde parten y para donde van. De esta manera, se crearía una política que optimice los transmilenios en funcionamiento y tambien evidenciar si es necesario tener mas líneas de transporte.

In [13]:
class red:

    #Para crear una red tendremos que definir los nodos que estaran, los lados (con capacidad y flujo), la fuente y el desague
    '''
    Atributos de la clase del grafo
    los lados se definen de la siguiente manera:
    [(1,2)] donde 1 es la cola y el 2 es la cabeza
    '''
    def __init__(self, nodos, lados,fuente,desague):
        self.n=nodos
        self.e=lados
        self.f=fuente
        self.d=desague
    
    '''
    Funcion para convertir una red en un grafo sin direcciones
    '''
    #Esta funcion nos permitira transformar el digrafo en un grafo sin direcciones
    def no_diri (self):
        lados_no_dirigidos=[]
        for i in self.e:
            lados_no_dirigidos.append(i[0])
            
        k=estaciones_transmilenio_no_dirigido(self.n,lados_no_dirigidos)
        return k
    
    '''
    Los nodos se caracterizan por de esa estacion de transmilenio sale y entra gente, por tal motivo con las siguientes funciones
    lo que deseo mostrar es como podemos definir cuantas personas entran y salen de una estación.
    '''
    # Funcion que nos dice que tanto flujo le sale a un nodo
    def f_mas(self,nodo):
        # Variable que empieza diciendo que sale 0 
        f=0
        # Recorro los lados del grafo
        for i in self.e:
            # Con este 'if' busco al nodo 
            if i[0][0]==nodo:
                j=i[1]
                f=f+j['ff']
        return f        
    #Funcion que dice que tanto flujo le entra a un nodo
    def f_menos(self,nodo):
        f=0
        for i in self.e:
            if i[0][1]==nodo:
                j=i[1]
                f=f+j['ff']
        return f
    # Funcion para definir la tolerancia de un camino f aumentador    
    def tolerancia(self,flujo):
        lista_excesos=[]
        for i in flujo:
            # Si el lado va en "direccion" el exceso sera la capacidad - el flujo
            if i[0][0]<i[0][1]:
                e=i[1]['cap']-i[1]['ff']
                lista_excesos.append(e)
            # De lo contrario el exceso de un lado que va en "contra" sera el mismo flujo, siempre y cuando este sea positivo
            elif i[0][0]>i[0][1] and i[1]['ff']>0:
                e=i[1]['ff']
                lista_excesos.append(e)
        #La tolerancia sera el minimo de los excesos del camino        
        tol=min(lista_excesos)
        return tol

    #Funcion para ver si un flujo es factible
    def flujo_factible(self):
        x=True
        lista_nodos=[]
        for j in self.n:
            #Miramos los nodos diferentes a la fuente y el desague
            if j!=self.f and j!=self.d:
                h=self.f_mas(j)
                k=self.f_menos(j)
                # Para estos nodos, lo que entra debe ser igual a lo que sale
                if h==k:
                    x= True
                else:
                    return False
        #Ahora para los lados la capacidad tiene que ser mayor o igual al flujo y este a su vez mayor o igual a cero
        for i in self.e:
            j=i[1]
            if j['cap']>=j['ff'] and j['ff']>=0 and x==True:
                return True
            else:
                return False

    #Funcion que nos permite ver si existe el lado en el camino f aumentador
    def existe_lado(self,flujo,k):
        for i in flujo:
            if k in i:
                return True
            else:
                return False

    #Algoritmo que nos permitira encontrar el maximo flujo y el corte minimo
    def ford_fulkerson(self,flujo):
        #Inicializamos S y R={s} (fuente)
        #Ademas de ello creamos variables que usaremos mas adelante
        s=[]
        r=[]
        r.append(self.f)
        #Se almacena una primera version del camino aumentador
        camino_aumentador=[]
        #los lados que no tendrian que hacer parte del camino f aumentador
        nodos_no=[]
        camino_aumentador.append(self.f)
        #Buscamos un nodo que este en R pero no en S
        for i in r:
            if self.d not in r and r!=s:
                if i not in s:
                    for k in flujo:
                        h=k[0]
                        n=k[1]
                        if i in h:
                            #Si este nodo es la cola de otro nodo se realiza lo sig
                            if i==h[0]:
                                #Si la cabeza no esta en s
                                if h[1] not in s:
                                    #Y la capacidad del arco que los une es mayor que el flujo, donde la cabeza no pertenece a r
                                    if n['cap']>n['ff'] and h[1] not in r:
                                        #Agregamos el nodo a R
                                        r.append(h[1])
                                        # Y tambien a la primera version del camino aumentador
                                        camino_aumentador.append(h[1])
                                    # En caso de que no cumpla la condicion ese lado no puede ir en el camino f aumentador
                                    elif n['cap']<=n['ff']:
                                        nodos_no.append([i,h[1]])
                            #Ahora si este nodo es la cabeza de otro nodo
                            elif i==h[1]:
                                #Si la cola no pertenece a S
                                if h[0] not in s:
                                    # Quiere decir que el lado va en direccion contraria por tanto mirarmos si el el flujo es positivo
                                    # Luego realizamos el mismo proceso de la otra condicion
                                    if n['ff']>0 and h[0] not in r:
                                        r.append(h[0])
                                        camino_aumentador.append(h[0])
                                    elif n['ff']<=0:
                                        nodos_no.append([h[0],i])
                    #Decimos que el nodo ya esta observado y lo agregamos a S                       
                    s.append(i)

        # Si despues de la iteracion el desague esta en R
        if self.d in r:
            #g sera el grafo no dirigida
            g=self.no_diri()
            g.crearvecindades()
            #Vecindades
            vec=g.vecindades
            #Todos los posibles caminos de la fuente al desague
            u=g.crear_caminos(self.f,self.d)
            caminos=list(u)
            #Basicamente eliminamos los caminos que no correspondan al grafo dirigido y tambien los que tengan los lados que no deseamos que esten el camino
            for i in nodos_no:
                x1=i[0]
                x2=i[1]
                for j in caminos:
                    for a in range(0,len(j)):
                        if (a+1)<len(j):
                            g1=j[a]
                            g2=j[a+1]
                            if g1==x1 and g2==x2:
                                caminos.remove(j)
                                
            # Para garantizar que sea aumentador, todos los nodos de este camino deben estar en la primera version del camino aumentador    
            for i in caminos:
                for h in i:
                    if h not in camino_aumentador:
                        caminos.remove(i)
            

                
            #Para tener el flujo maximo requerimos cambiar el flujo dada una nueva tolerancia    
            f_aumentador=[]
            for u in caminos:
                for i in u:
                    for j in u:
                        for b in flujo:
                            if(i,j)==b[0]:
                                f_aumentador.append(b)

            # Se encuentra la tolerancia del camino y modificamos el flujo dada las reglas
            x=self.tolerancia(f_aumentador)
            print ('tolerancia')
            print (x)
            for b in f_aumentador:
                if b[0][0]<b[0][1]:
                    b[1]['ff']=b[1]['ff']+x
                elif b[0][0]>b[0][1]:
                    b[1]['ff']=b[1]['ff']-x
            
            nodos=[]
            for a in f_aumentador:
                nodos.append(a[0])

            for b in flujo:
                if b[0] not in nodos:
                    f_aumentador.append(b)
            #Retornamos el max flujo
            return f_aumentador
        
        # Si S=R retornamos el corte minimo de nodos       
        elif s==r:
            ## Corte de nodos
            return s
            

In [35]:
k=red([1,2,3,4,5,6],[[(1,2),{'cap':200,'ff':100}],[(1,3),{'cap':200,'ff':0}],\
                     [(2,5),{'cap':100,'ff':100}],\
                     [(5,3),{'cap':100,'ff':100}],[(3,4),{'cap':100,'ff':100}],\
                     [(4,6),{'cap':200,'ff':100}],[(5,6),{'cap':200,'ff':0}]],1,6)

In [36]:
x=k.ford_fulkerson(k.e)
print ('flujo maximo')
print (x)
j=k.ford_fulkerson(x)
print ('corte')
print (j)

tolerancia
100
flujo maximo
[[(1, 3), {'cap': 200, 'ff': 100}], [(5, 3), {'cap': 100, 'ff': 0}], [(5, 6), {'cap': 200, 'ff': 100}], [(1, 2), {'cap': 200, 'ff': 100}], [(2, 5), {'cap': 100, 'ff': 100}], [(3, 4), {'cap': 100, 'ff': 100}], [(4, 6), {'cap': 200, 'ff': 100}]]
corte
[1, 3, 2]


Base de datos: Encuesta de movilidad


Creo las diferentes líneas de transmilenio 

In [ ]:
# Calle 26
'''
Portal dorado=1
Modelia=2
Normandia=3
AvRojas=4
Tiempo=5
Salitre=6
CAN=7
Gobernacion=8
QuintaParedes=9
Corferias=10
CiuUniver=11
ConcejoBogota=12
CentroMemoria=13
Universidades=14
'''
estaciones_26=[()]



In [26]:
pkl_file = open('viajes_df.pkl', 'rb')
viajes = pickle.load(pkl_file)
pkl_file.close()

filtros = viajes.MEDIO_PREDOMINANTE.unique()
viajes["INICIO"] = viajes.apply(lambda row: row["HORA_INICIO"][0:2]+":00",axis=1)
viajes["FIN"] = viajes.apply(lambda row: row["HORA_FIN"][0:2]+":00",axis=1)
locations = viajes
locations = locations.drop(locations[locations.ZAT_ORIGEN != locations.ZAT_HOGAR].index)
locations = locations.groupby("ZAT_HOGAR").first()
locations = locations.loc[:,["BARRIO","longitude_o", "ZAT_ORIGEN",
                                 "latitude_o"]]
    
viajes['BARRIO_D'] = viajes['ZAT_DESTINO'].map(locations.set_index('ZAT_ORIGEN')['BARRIO'])
viajes['BARRIO'] = viajes['ZAT_ORIGEN'].map(locations.set_index('ZAT_ORIGEN')['BARRIO'])
viajes["FRANJA"] = np.where((viajes["INICIO"]>="00:00") & (viajes["INICIO"]<="05:00"), "Madrugada", "Hola")
viajes["FRANJA"] = np.where((viajes["INICIO"]>="06:00") & (viajes["INICIO"]<="12:00"), "Mañana", viajes["FRANJA"])
viajes["FRANJA"] = np.where((viajes["INICIO"]>="13:00") & (viajes["INICIO"]<="18:00"), "Tarde", viajes["FRANJA"])
viajes["FRANJA"] = np.where((viajes["INICIO"]>="19:00") & (viajes["INICIO"]<="23:00"), "Noche", viajes["FRANJA"])

In [34]:
viajes.columns[1]

'NUMERO_PERSONA'

In [30]:
# Control del tamaño de la figura del mapa
fig, ax = plt.subplots(figsize=(10, 10))
 
# Control del título y los ejes
ax.set_title('Natalidad por Provincias en España, 2018', 
             pad = 20, 
             fontdict={'fontsize':20, 'color': '#4873ab'})
ax.set_xlabel('longitude_o')
ax.set_ylabel('latitude_o')
 
# Mostrar el mapa finalizado
map_data.plot(column='NAT2018', cmap='plasma', ax=ax, zorder=5)


0         4.611812
1         4.652134
2         4.611812
3         4.615805
4         4.659513
5         4.695778
6         4.659513
7         4.652723
8         4.611005
9         4.634490
10        4.611005
11        4.634490
12        4.611005
13        4.615574
14        4.609918
15        4.615412
16        4.609918
17        4.676304
18        4.609918
19        4.688711
20        4.616660
21        4.638118
22        4.616660
23        4.577718
24        4.638863
25        4.701238
26        4.638863
27        4.751013
28        4.620523
29        4.566569
            ...   
144920    4.811361
144921    4.700817
144922    4.811361
144923    4.811361
144924    4.811361
144925    4.811361
144926    4.811361
144927    4.811361
144928    4.811361
144929    4.811361
144930    4.811361
144931    4.811361
144932    4.811361
144933    4.811361
144934    4.811361
144935    4.811361
144936    4.811361
144937    4.811361
144938    4.811361
144939    4.731808
144940    4.811361
144941    4.